# RAG 평가하기

### 선수 조건
- Amazon Bedrock Knowledge base 를 먼저 생성 해야 합니다.
    - 참조 블로그: [Amazon Bedrock Knowledge base로 30분 만에 멀티모달 RAG 챗봇 구축하기 실전 가이드](https://aws.amazon.com/ko/blogs/tech/practical-guide-for-bedrock-kb-multimodal-chatbot/)
    - data: ../data/241231-Sonnet-recipe.pdf 문서를 사용하세요.

---    

## 0. 환경 설정

In [4]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)


module_path = ".."
add_python_path(module_path)


from libs.notebook_util import (
    pretty_print_df, convert_to_list, format_text,
    print_formatted_text, generate_answer,
    show_sample_row_eval,
    merge_dataset_and_results
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
python path: /home/sagemaker-user/rag-evaluation-bedrock already exists
sys.path:  ['/home/sagemaker-user/rag-evaluation-bedrock/notebook_kr', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python310.zip', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/lib-dynload', '', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/site-packages', '/home/sagemaker-user/rag-evaluation-bedrock']


## 1. 합성 데이터 로딩
    


In [5]:
import pandas as pd

df = pd.read_json("../data/sample_qa_dataset.jsonl", lines=True)
print(df.shape)
df.head()

(879, 4)


,question,ground_truth,question_type,contexts
0,"한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요...","한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1....",complex,"볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다..."
1,타진은 어떤 나라의 요리이며 어떻게 만드나요?,타진은 모로코 요리입니다. 원뿔 모양의 뚜껑이 있는 특별한 그릇에서 양고기나 닭고기...,simple,볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...
2,"한국, 이탈리아, 태국의 요리를 각각 하나씩 선택하여, 이 요리들의 공통점과 차이점...","한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n공통...",complex,"볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다..."
3,다양한 국가의 요리 중 유사한 조리 방식이나 재료를 사용하지만 문화적 특성에 따라 ...,여러 국가의 요리 중 유사한 조리 방식이나 재료를 사용하지만 문화적 특성에 따라 다...,complex,볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...
4,김치볶음밥에 마지막에 무엇을 넣어 고소한 향을 더하나요?,김치볶음밥에 마지막에 참기름을 둘러 고소한 향을 더합니다.,simple,"볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다..."


In [6]:
# sample_df = df.iloc[[1]]
pretty_print_df(df.iloc[[0]])    
# df.head(1).iloc[[0]].contexts.iloc[0]

question:
한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요.
특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.
ground_truth:
한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.
1.
주요 재료: - 김치볶음밥: 밥, 김치, 참기름 - 리소토: 아르보리오 쌀, 화이트와인, 버터, 육수 - 팟타이: 쌀국수, 새우, 땅콩가루 2.
조리 방법: 유사점: 세 요리 모두 볶는 조리법을 사용합니다.
차이점: - 김치볶음밥: 빠르게 고열로 볶아 만듭니다.
- 리소토: 천천히 육수를 넣어가며 졸이듯이 조리합니다.
- 팟타이: 중간 불에서 재료를 순서대로 넣어가며 볶습니다.
3.
문화적 특성: - 김치볶음밥: 한국의 발효 문화를 대표하는 김치를 활용하여 간편하게 만드는 요리로, 한국인의 실용적이고 빠른 식문화를 반영합니다.
- 리소토: 이탈리아의 여유로운 식문화를 반영하여 천천히 정성을 들여 만듭니다.
와인을 사용하는 것은 이탈리아의 와인 문화를 보여줍니다.
- 팟타이: 태국의 길거리 음식 문화를 대표하며, 다양한 재료와 맛(새콤, 달콤, 고소함)의 조화를 중시하는 태국 요리의 특징을 잘 보여줍니다.
이처럼 세 요리는 모두 볶음 요리이지만, 각 나라의 고유한 식재료와 조리 방식, 그리고 식문화적 특성을 뚜렷하게 반영하고 있습니다.
question_type:
complex
contexts:
볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다.
제육볶음은 돼지고기를 고 추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어 영양을 높입니다.
낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게 만드는 요리입니다.
양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고 베이컨이나 새우를 더해 고소하게 완성합니다

## 2. Test Data Set 생성

#### Test Dataset 갯수 지정

In [7]:
subset_length=2
df.head(subset_length)

,question,ground_truth,question_type,contexts
0,"한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요...","한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1....",complex,"볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다..."
1,타진은 어떤 나라의 요리이며 어떻게 만드나요?,타진은 모로코 요리입니다. 원뿔 모양의 뚜껑이 있는 특별한 그릇에서 양고기나 닭고기...,simple,볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...


In [8]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df.head(subset_length))
test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map: 100%|██████████| 2/2 [00:00<00:00, 537.28 examples/s]

Error: invalid syntax (<unknown>, line 1)
Error: invalid syntax (<unknown>, line 1)
Dataset({
    features: ['question', 'ground_truth', 'question_type', 'contexts'],
    num_rows: 2
})


In [9]:
test_dataset[0]

{'question': '한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요. 특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.',
 'ground_truth': '한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1. 주요 재료:\n- 김치볶음밥: 밥, 김치, 참기름\n- 리소토: 아르보리오 쌀, 화이트와인, 버터, 육수\n- 팟타이: 쌀국수, 새우, 땅콩가루\n\n2. 조리 방법:\n유사점: 세 요리 모두 볶는 조리법을 사용합니다.\n차이점: \n- 김치볶음밥: 빠르게 고열로 볶아 만듭니다.\n- 리소토: 천천히 육수를 넣어가며 졸이듯이 조리합니다.\n- 팟타이: 중간 불에서 재료를 순서대로 넣어가며 볶습니다.\n\n3. 문화적 특성:\n- 김치볶음밥: 한국의 발효 문화를 대표하는 김치를 활용하여 간편하게 만드는 요리로, 한국인의 실용적이고 빠른 식문화를 반영합니다.\n- 리소토: 이탈리아의 여유로운 식문화를 반영하여 천천히 정성을 들여 만듭니다. 와인을 사용하는 것은 이탈리아의 와인 문화를 보여줍니다.\n- 팟타이: 태국의 길거리 음식 문화를 대표하며, 다양한 재료와 맛(새콤, 달콤, 고소함)의 조화를 중시하는 태국 요리의 특징을 잘 보여줍니다.\n\n이처럼 세 요리는 모두 볶음 요리이지만, 각 나라의 고유한 식재료와 조리 방식, 그리고 식문화적 특성을 뚜렷하게 반영하고 있습니다.',
 'question_type': 'complex',
 'contexts': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적

# 3. Amazon Bedrock KB 생성

이 테스트 데이터셋은 문서 검색과 응답 생성을 결합하는 RAG 파이프라인에서 실제 쿼리를 시뮬레이션하는 데 사용됩니다.

여기서는 RAG 구성의 일부로 Amazon Bedrock의 KnowledgeBase에 대한 기본 설정을 사용하고 있습니다.

_1. 아래 코드로 테스트하려면 Amazon Bedrock 콘솔에서 KnowledgeBase가 사전 구성되어 있어야 합니다_

_2. 평가하고자 하는 특정 RAG 파이프라인이 있다면 아래 셀을 적절히 수정하시기 바랍니다_

## 3.1 컨텍스트를 KB로 부터 가져오기

In [7]:
# RAG implementation sample 1 (Replace with RAG pipeline for evaluation)
from libs.bedrock_kb_util import context_retrieval_from_kb

amazon_kb_id = 'RAILTN1YDB'

question = test_dataset[0]['question']
# question = test_dataset[1]['question']
print("Question:\n", question)
search_result = context_retrieval_from_kb(question, 3, 'us-west-2', amazon_kb_id, 'SEMANTIC')
# print("search_result[0]:", search_result[0])

contexts = "\n--\n".join([result['content'] for result in search_result])
print("Context from Amazon KB:")
print(format_text(contexts))
# print_formatted_text(contexts)

Question:
 한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요. 특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.


Context from Amazon KB:
볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다.
제육볶음은 돼지고기를 고 추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어 영양을 높입니다.
낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게 만드는 요리입니다.
양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고 베이컨이나 새우를 더해 고소하게 완성합니다.
토마토파스타는 토마토소스에 마늘 과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다.
미트소스스파게티는 다 진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.
일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바 삭하게 튀겨내고 특제 소스를 곁들입니다.
카레라이스는 당근, 감자, 양파를 카 레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다.
오야코동은 닭고 기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.
중식으로는 마파두부가 있는데, 두부와 다진 고기를 매운 두반장 소스에 볶아 만 듭니다.
깐풍기는 닭고기를 튀겨서 매콤달콤한 소스를 絡めて만드는 요리이고, 탕 수육은 돼지고기를 튀겨서 새콤달콤한 소스를 부어 먹습니다.
태국 요리로는 팟타이가 유명한데, 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려 고소한 맛을 냅니다.
그린커리는 코코넛밀크와 향신료를 넣어 만든 커리에 닭고기 와 가지를 넣어 끓입니다.
톰얌쿵은 새우와 버섯을 레몬그라스, 카피르라임 등의 향신료와 함께 시큼매콤하게 끓이는 수프입니다.
베트남 요리에서는 쌀국수인 포가 대표적인데, 소고기 육수에 쌀국수와 고기를 넣 고 고수, 숙주나물을 곁들여 먹습니다.
반미는 바게트빵에 각종 채소와 고기를 넣어 만드는 샌드위치이며, 분짜는 구운 돼지고기와 쌀국수를 느억맘 소스에 찍어 먹는 요리입니다.
이탈리아 요리로 더 들어가보면, 리소토는 아르보리오 쌀을 화이트와인

## 3.2 KB로 부터 Context를 LLM 을 통한 답변 생성
여기서는 검색된 컨텍스트를 기반으로 답변을 생성하고 있습니다.

In [8]:
import boto3
from botocore.config import Config

# model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
region = 'us-west-2'

retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", config=retry_config)
answer = generate_answer(question, contexts, boto3_client, model_id)
print("Question: \n", question)
print("Answer: ")
print_formatted_text(answer)


Question: 
 한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요. 특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.
Answer: 
분석해보겠습니다.

한국: 제육볶음
- 주요 재료: 돼지고기, 양파, 당근
- 조리 방법: 고추장 양념에 버무려 매콤달콤하게 볶음

이탈리아: 미트소스스파게티
- 주요 재료: 소고기, 토마토소스
- 조리 방법: 다진 쇠고기를 토마토소스와 함께 오래 끓임

태국: 팟타이
- 주요 재료: 쌀국수, 새우, 땅콩가루
- 조리 방법: 쌀국수를 새우와 함께 볶고 땅콩가루 뿌림

공통점:
- 모두 볶음 요리
- 고기를 주재료로 사용
- 강한 양념이나 소스 활용

차이점:
- 한국: 매콤달콤한 고추장 특성
- 이탈리아: 오래 끓여 깊은 맛 강조
- 태국: 땅콩가루로 고소함 추가

각 요리는 해당 나라의 독특한 조리 문화와 입맛을 잘 반영하고 있습니다.


# 4. Text Dataset 의 모든 샘플 질문에 대해 KB로 부터 Context 를 얻고 LLM 통해 답변 얻기

이 섹션에서는 컨텍스트 검색부터 답변 생성까지 전체 파이프라인을 샘플 질문들에 대해 실행합니다.

### 4.1 파라미터 설정

In [9]:
kb_region = 'us-west-2'
# model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
kb_id = amazon_kb_id
top_k = 3

### 4.2 KB 로 부터 Context 얻고, LLM 답변 생성

In [10]:
from time import sleep

def process_item(item):
    sleep(5)  # Prevent throttling
    question = item['question']
    search_result = context_retrieval_from_kb(question, top_k, kb_region, kb_id, 'SEMANTIC')

    contexts = [result['content'] for result in search_result]
    answer = generate_answer(question, "\n--\n".join(contexts), boto3_client, model_id)

    return {
        'question': item['question'],
        'ground_truth': item['ground_truth'],
        'original_contexts': item['contexts'],
        'retrieved_contexts': contexts,
        'answer': answer
    }

updated_dataset = test_dataset.map(process_item, remove_columns=test_dataset.column_names)  # 기존의 모든 컬럼 제거)

Parameter 'function'=<function process_item at 0x7f7f8be9fd00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map: 100%|██████████| 2/2 [00:22<00:00, 11.15s/ examples]


In [11]:
updated_dataset

Dataset({
    features: ['question', 'ground_truth', 'original_contexts', 'retrieved_contexts', 'answer'],
    num_rows: 2
})

# 5. RAGAS Rag 평가 수행

## 5.1. RAGAS 래핑 모델 생성

In [12]:
# model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
# emb_model_id = "us.amazon.nova-micro-v1:0"
emb_model_id = "amazon.titan-embed-text-v2:0"

In [13]:
import boto3
from datasets import Dataset

from langchain_aws import ChatBedrockConverse
from ragas import evaluate

# Bedrock 클라이언트 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatBedrockConverse(
    model= model_id, 
    client=bedrock_client,
))

from langchain_community.embeddings import BedrockEmbeddings
# from ragas.embeddings import LangchainEmbeddingWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Bedrock Embeddings 설정
base_embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id= emb_model_id
)

# RAGAS Wrapper로 감싸기
evaluator_embeddings = LangchainEmbeddingsWrapper(base_embeddings)

/tmp/ipykernel_17215/1375331539.py:25: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  base_embeddings = BedrockEmbeddings(


### 5.2. 평가용 데이터셋 생성

In [14]:
def prepare_ragas_evaluation_dataset(example):
    return {
        "question": example["question"],
        "answer": example["answer"],
        "contexts": example["retrieved_contexts"],
        "ground_truth": example["ground_truth"]
    }

data = [{
    "question": "파이썬이란 무엇인가요?",
    "ground_truth": "파이썬은 쉽고 간결한 프로그래밍 언어입니다.",
    "retrieved_contexts": ["파이썬은 프로그래밍 언어입니다.", "파이썬은 읽기 쉽고 간결합니다."],
    "generated_answer": "파이썬은 읽기 쉽고 간결한 프로그래밍 언어입니다."
}]
# dataset = Dataset.from_list(data)


# 2. RAGAS 평가용 데이터셋 포맷으로 변환
eval_dataset = updated_dataset.map(
    prepare_ragas_evaluation_dataset,
    remove_columns=updated_dataset.column_names
)
eval_dataset


Map: 100%|██████████| 2/2 [00:00<00:00, 566.72 examples/s]


Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 2
})

### 5.2. 평가 메트릭으로 정의 후 실행

In [15]:
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextRecall,
    ContextPrecision
)
# 먼저 Faithfulness만 테스트
try:
    metrics = [
                Faithfulness(llm=evaluator_llm),
                AnswerRelevancy(llm=evaluator_llm, embeddings= evaluator_embeddings),
                ContextRecall(llm=evaluator_llm),
                ContextPrecision(llm=evaluator_llm),
    ]
    
    # 평가 실행a
    results = evaluate(
        eval_dataset,
        metrics=metrics,
        # show_individual_scores=True  # 이 부분이 추가됨
    )
    
except Exception as e:
    print(f"Error occurred: {str(e)}")
    print(f"Error type: {type(e)}")

results

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 8/8 [00:16<00:00,  2.10s/it]


{'faithfulness': 0.6667, 'answer_relevancy': 0.3686, 'context_recall': 0.8000, 'context_precision': 1.0000}

In [16]:
result_df = results.to_pandas()
result_df

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision
0,"한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요...","[볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다...","제공된 맥락을 바탕으로 한국의 김치볶음밥, 이탈리아의 크림파스타, 태국의 팟타이를 ...","한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1....",0.333333,0.434817,0.6,1.0
1,타진은 어떤 나라의 요리이며 어떻게 만드나요?,[끼슈는 버터가 듬뿍 들어간 파이 반죽 위 에 달걀과 크림을 섞은 커스터드를 붓고 ...,타진은 모로코 요리입니다. 원뿔 모양의 뚜껑이 있는 특별한 그릇에서 양고기나 닭고기...,타진은 모로코 요리입니다. 원뿔 모양의 뚜껑이 있는 특별한 그릇에서 양고기나 닭고기...,1.000000,0.302435,1.0,1.0


### 5.3. 각각의 결과 보기

In [17]:

show_sample_row_eval(result_df, row=0)


## Question: 
 한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요. 특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.
## Ground_truth: 
 한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.

1. 주요 재료:
- 김치볶음밥: 밥, 김치, 참기름
- 리소토: 아르보리오 쌀, 화이트와인, 버터, 육수
- 팟타이: 쌀국수, 새우, 땅콩가루

2. 조리 방법:
유사점: 세 요리 모두 볶는 조리법을 사용합니다.
차이점: 
- 김치볶음밥: 빠르게 고열로 볶아 만듭니다.
- 리소토: 천천히 육수를 넣어가며 졸이듯이 조리합니다.
- 팟타이: 중간 불에서 재료를 순서대로 넣어가며 볶습니다.

3. 문화적 특성:
- 김치볶음밥: 한국의 발효 문화를 대표하는 김치를 활용하여 간편하게 만드는 요리로, 한국인의 실용적이고 빠른 식문화를 반영합니다.
- 리소토: 이탈리아의 여유로운 식문화를 반영하여 천천히 정성을 들여 만듭니다. 와인을 사용하는 것은 이탈리아의 와인 문화를 보여줍니다.
- 팟타이: 태국의 길거리 음식 문화를 대표하며, 다양한 재료와 맛(새콤, 달콤, 고소함)의 조화를 중시하는 태국 요리의 특징을 잘 보여줍니다.

이처럼 세 요리는 모두 볶음 요리이지만, 각 나라의 고유한 식재료와 조리 방식, 그리고 식문화적 특성을 뚜렷하게 반영하고 있습니다.
## Generated Answer: 
 제공된 맥락을 바탕으로 한국의 김치볶음밥, 이탈리아의 크림파스타, 태국의 팟타이를 비교해보겠습니다.

주요 재료:
- 김치볶음밥: 밥, 김치, 참기름
- 크림파스타: 파스타, 생크림, 마늘, 양파, 베이컨 또는 새우
- 팟타이: 쌀국수, 새우, 땅콩가루

조리 방법:
- 공통점: 모두 볶음 조리법을 사용하며, 여러 재료를 함께 조리
- 차이점: 
  * 김치볶음밥은 단순한 재료로 간단히 조리
  * 크림파스타는 소스 만들기에 더 복잡한 과

# 6. Custom RAG Evaluation 수행

이제 다양한 지표를 기반으로 시스템을 평가하겠습니다.

자세한 구현 내용은 `libs/custom_ragas.py` 파일을 참조하시기 바랍니다.

이 스크립트에는 정확도와 관련성 등 여러 측면에서 RAG 파이프라인의 성능을 평가하는데 사용되는 구체적인 평가 기준이 포함되어 있습니다.

### 6.1. 평가용 데이터셋 생성

In [18]:
from libs.custom_ragas import (
    evaluate,
    AnswerRelevancy, 
    Faithfulness, 
    ContextRecall,
    ContextPrecision
)

# llm_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
llm_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
# llm_id = "us.amazon.nova-micro-v1:0"
emb_id = "amazon.titan-embed-text-v2:0"
region = "us-west-2"

# metrics = [Faithfulness]
metrics = [AnswerRelevancy, Faithfulness, ContextRecall, ContextPrecision]

def map_dataset(example):
    return {
        "user_input": example["question"],
        "retrieved_contexts": example["retrieved_contexts"],
        "referenced_contexts": example["original_contexts"],
        "response": example["answer"],
        "reference": example["ground_truth"]
    }

dataset = updated_dataset.map(map_dataset, remove_columns=updated_dataset.column_names)  # 기존의 모든 컬럼 제거))


Map: 100%|██████████| 2/2 [00:00<00:00, 502.31 examples/s]


In [19]:
dataset

Dataset({
    features: ['retrieved_contexts', 'user_input', 'referenced_contexts', 'response', 'reference'],
    num_rows: 2
})

### 6.2. 평가 메트릭으로 정의 후 실행

In [20]:
results = evaluate(dataset, metrics, llm_id, emb_id, region)
# results = evaluate(dataset, metrics, llm_id, emb_id, region, verbose=True)

print("Average Scores:")
print(results['average_scores'])

print("\nDetailed Results:")
for row in results['detailed_results']:
    print(row)

AnswerRelevancy - Row 1: Score = 0.541079385042994
Faithfulness - Row 1: Score = 1.0
ContextRecall - Row 1: Score = 0.8125
ContextPrecision - Row 1: Score = 0.9999999999
AnswerRelevancy - Row 2: Score = 0.3223272047270477
Faithfulness - Row 2: Score = 1.0
ContextRecall - Row 2: Score = 1.0
ContextPrecision - Row 2: Score = 0.9999999999
Average Scores:
{'AnswerRelevancy': 0.43170329488502085, 'Faithfulness': 1.0, 'ContextRecall': 0.90625, 'ContextPrecision': 0.9999999999}

Detailed Results:
{'row': 1, 'AnswerRelevancy': 0.541079385042994, 'Faithfulness': 1.0, 'ContextRecall': 0.8125, 'ContextPrecision': 0.9999999999}
{'row': 2, 'AnswerRelevancy': 0.3223272047270477, 'Faithfulness': 1.0, 'ContextRecall': 1.0, 'ContextPrecision': 0.9999999999}


### 6.3. 결과를 JSON 에 저장

In [21]:
import json
import json

def save_json_results(results, json_filename="../data/sample_ragas_result.json"):
    json_results = {
        'average_scores': results['average_scores'],
        'detailed_results': results['detailed_results']
    }

    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_results, f, ensure_ascii=False, indent=4)

    print(f"Results saved to {json_filename}")
    print(json_results)
    
    return json_results

json_filename="../data/sample_ragas_result.json"
saved_results = save_json_results(results, json_filename)

saved_results = save_json_results(results)
# Or with a custom filename:
# saved_results = save_json_results(results, "custom_path/results.json")


Results saved to ../data/sample_ragas_result.json
{'average_scores': {'AnswerRelevancy': 0.43170329488502085, 'Faithfulness': 1.0, 'ContextRecall': 0.90625, 'ContextPrecision': 0.9999999999}, 'detailed_results': [{'row': 1, 'AnswerRelevancy': 0.541079385042994, 'Faithfulness': 1.0, 'ContextRecall': 0.8125, 'ContextPrecision': 0.9999999999}, {'row': 2, 'AnswerRelevancy': 0.3223272047270477, 'Faithfulness': 1.0, 'ContextRecall': 1.0, 'ContextPrecision': 0.9999999999}]}
Results saved to ../data/sample_ragas_result.json
{'average_scores': {'AnswerRelevancy': 0.43170329488502085, 'Faithfulness': 1.0, 'ContextRecall': 0.90625, 'ContextPrecision': 0.9999999999}, 'detailed_results': [{'row': 1, 'AnswerRelevancy': 0.541079385042994, 'Faithfulness': 1.0, 'ContextRecall': 0.8125, 'ContextPrecision': 0.9999999999}, {'row': 2, 'AnswerRelevancy': 0.3223272047270477, 'Faithfulness': 1.0, 'ContextRecall': 1.0, 'ContextPrecision': 0.9999999999}]}


### 6.4 최종 결과 보이기

In [22]:
# 사용 예시
results_path = json_filename
merged_df = merge_dataset_and_results(updated_dataset, results_path)
print("Average Scores:")
print(results['average_scores'])

merged_df


Average Scores:
{'AnswerRelevancy': 0.43170329488502085, 'Faithfulness': 1.0, 'ContextRecall': 0.90625, 'ContextPrecision': 0.9999999999}


,question,ground_truth,original_contexts,retrieved_contexts,answer,AnswerRelevancy,Faithfulness,ContextRecall,ContextPrecision
0,"한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요...","한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1....","볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다...","[볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다...","제공된 맥락을 바탕으로 한국의 김치볶음밥, 이탈리아의 크림파스타, 태국의 팟타이를 ...",0.541079,1.0,0.8125,1.0
1,타진은 어떤 나라의 요리이며 어떻게 만드나요?,타진은 모로코 요리입니다. 원뿔 모양의 뚜껑이 있는 특별한 그릇에서 양고기나 닭고기...,볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...,[끼슈는 버터가 듬뿍 들어간 파이 반죽 위 에 달걀과 크림을 섞은 커스터드를 붓고 ...,타진은 모로코 요리입니다. 원뿔 모양의 뚜껑이 있는 특별한 그릇에서 양고기나 닭고기...,0.322327,1.0,1.0000,1.0
